In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
import scipy.stats

# Load data
df1 = pd.read_csv('~/data/BAH_PRS/data_and_code/pato_delong_test_dat.csv', sep=',', header=0)
df1

,BAH,Age,BMI,WC,SBP,DBP,TG,LDL,FBG,PRC,...,TG_bin,LDL_bin,FBG_bin,Renin_bin,PAC_bin_10,PAC_bin_20,PAC_bin_30,ARR_bin_15,ARR_bin_20,ARR_bin_25
0,0,0.000263,2.038464,1.831030,-0.493333,0.730303,0.008341,0.453455,-0.839104,-0.110192,...,0,1,0,0,1,1,1,1,1,0
1,0,-0.107371,0.323885,-0.368836,-0.493333,-2.390146,-0.154527,-0.822605,-0.889204,0.505636,...,0,0,0,0,1,1,0,0,0,0
2,0,0.323165,-0.672425,-0.840236,-2.219241,-0.517876,-0.453117,-1.155490,-0.087592,0.343179,...,0,0,0,0,1,1,0,0,0,0
3,0,-1.506614,-0.232195,-0.054569,0.197030,-0.517876,0.632666,-0.794864,0.714020,0.509415,...,1,0,0,0,1,1,0,0,0,0
4,0,0.430799,1.899444,1.359630,-0.493333,-3.139054,0.429082,0.508936,0.513617,0.112715,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1632,0,0.538433,-0.880955,-0.054569,0.197030,1.354393,-0.425973,-0.156834,0.263113,-0.091302,...,0,1,0,0,1,0,0,0,0,0
1633,0,0.430799,-1.112655,-0.290269,0.197030,-0.517876,0.944829,-0.767124,-0.688801,-0.658016,...,1,0,0,1,0,0,0,0,0,0
1634,0,-2.044784,-0.417555,-0.211703,-0.493333,-0.517876,-0.290250,-0.767124,-0.488398,-0.695797,...,0,0,0,1,0,0,0,0,0,0
1635,0,0.430799,1.459214,0.652530,-0.493333,0.106213,-0.588840,-1.460635,-0.989406,-0.242426,...,0,0,0,0,1,1,0,1,0,0


In [2]:
#calculate 95% CI using bootstrapping
def bootstrap_auc_ci(y_true, y_proba, n_bootstraps=1000, confidence_level=0.95):
    y_true = np.array(y_true)
    y_proba = np.array(y_proba)
    
    auc_scores = []
    for _ in range(n_bootstraps):
        # Resample with replacement
        indices = np.random.choice(len(y_true), len(y_true), replace=True)
        if len(np.unique(y_true[indices])) < 2:
            continue  # Skip if only one class is present
        auc = roc_auc_score(y_true[indices], y_proba[indices])
        auc_scores.append(auc)
    # Calculate confidence interval
    lower_bound = np.percentile(auc_scores, (1 - confidence_level) / 2 * 100)
    upper_bound = np.percentile(auc_scores, (1 + confidence_level) / 2 * 100)
    return lower_bound, upper_bound

# DeLong test
def delong_test(y_true, proba1, proba2):
    """
    Perform the DeLong test to compare the AUCs of two models.

    Args:
        y_true (array-like): True binary labels (0 or 1).
        proba1 (array-like): Predicted probabilities from model 1.
        proba2 (array-like): Predicted probabilities from model 2.

    Returns:
        z_score (float): Z-score for the difference in AUCs.
        p_value (float): Two-tailed p-value for the difference in AUCs.
    """

    def compute_midrank(x):
        """
        Compute the midrank of elements in the array x.

        Args:
            x (array-like): Input array.

        Returns:
            T2 (np.ndarray): Midrank values for the input array.
        """
        J = np.argsort(x)  # Indices that would sort the array
        Z = x[J]  # Sorted array
        N = len(x)  # Length of the array
        T = np.zeros(N, dtype=np.float64)  # Initialize midrank array

        # Compute midranks for tied values
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:  # Find the range of tied values
                j += 1
            T[i:j] = 0.5 * (i + j - 1)  # Assign midrank to tied values
            i = j

        # Reorder midranks to match the original array
        T2 = np.empty(N, dtype=np.float64)
        T2[J] = T + 1  # Add 1 to match R's rank convention
        return T2

    def compute_ground_truth_statistics(true):
        """
        Compute statistics needed for the DeLong test.

        Args:
            true (np.ndarray): True binary labels (0 or 1).

        Returns:
            order (np.ndarray): Indices that sort the true labels in descending order.
            label_1_count (int): Number of positive examples (label 1).
        """
        assert np.array_equal(np.unique(true), [0, 1]), "Ground truth must be binary."
        order = (-true).argsort()  # Sort indices in descending order of true labels
        label_1_count = int(true.sum())  # Count of positive examples
        return order, label_1_count

    # Ensure y_true is a numpy array
    true = np.array(y_true)

    # Compute ground truth statistics
    order, label_1_count = compute_ground_truth_statistics(true)

    # Stack probabilities and sort according to ground truth order
    sorted_probs = np.vstack((np.array(proba1), np.array(proba2)))[:, order]

    # Initialize arrays for midranks
    m = label_1_count  # Number of positive examples
    n = sorted_probs.shape[1] - m  # Number of negative examples
    k = sorted_probs.shape[0]  # Number of models (2 in this case)
    tx, ty, tz = [np.empty([k, size], dtype=np.float64) for size in [m, n, m + n]]

    # Compute midranks for positive, negative, and all examples
    for r in range(k):
        positive_examples = sorted_probs[r, :m]  # Probabilities for positive examples
        negative_examples = sorted_probs[r, m:]  # Probabilities for negative examples
        tx[r, :], ty[r, :], tz[r, :] = [
            compute_midrank(examples) for examples in [positive_examples, negative_examples, sorted_probs[r, :]]
        ]

    # Compute AUCs for each model
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n

    # Compute covariance components
    v01 = (tz[:, :m] - tx[:, :]) / n  # Covariance between positive and negative examples
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m  # Covariance between negative and positive examples

    # Compute the DeLong covariance matrix
    sx = np.cov(v01)  # Covariance for positive examples
    sy = np.cov(v10)  # Covariance for negative examples
    delongcov = sx / m + sy / n  # Combined covariance matrix

    # Compute the Z-score and p-value
    l = np.array([[1, -1]])  # Contrast vector for comparing two models
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, delongcov), l.T)).flatten()  # Z-score
    p_value = scipy.stats.norm.sf(abs(z)) * 2  # Two-tailed p-value

    # Return the Z-score and p-value
    z_score = -z[0].item()  # Extract Z-score as a float
    p_value = p_value[0].item()  # Extract p-value as a float

    return z_score, p_value

In [3]:
# Define the columns to loop through
columns_to_loop = list(range(1, 4)) + list(range(15, 27)) + [10, 11]

# Split data for model 1 (trained on multiple features)
X_train, X_test, y_train, y_test = train_test_split(df1.iloc[:, 1:12], df1.iloc[:, 0], test_size=0.3, random_state=49)

# Train model 1 (Gaussian Naive Bayes)
model1 = GaussianNB()
model1.fit(X_train, y_train)
proba1 = model1.predict_proba(X_test)[:, 1]  # Probabilities for class 1

# Initialize a list to store results
results = []

# Set random seed for reproducibility
np.random.seed(42)
# Loop through each feature for model2
for col in columns_to_loop:
    # Split data for model2 (one feature at a time)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(df1.iloc[:, col:col+1], df1.iloc[:, 0], test_size=0.3, random_state=49)
    
    # Train model2 (Logistic Regression)
    model2 = LogisticRegression()
    model2.fit(X_train2, y_train2)
    proba2 = model2.predict_proba(X_test2)[:, 1]  # Probabilities for class 1
    
    # Calculate AUC and 95% CI for model2
    auc2 = roc_auc_score(y_test2, proba2)
    ci2_lower, ci2_upper = bootstrap_auc_ci(y_test2, proba2)
    
    # Perform DeLong test to compare model1 and model2
    z_score, p_value = delong_test(y_test, proba1, proba2)
    
    # Format AUC and CI as a string
    auc_ci = f"{auc2:.2f}({ci2_lower:.3f},{ci2_upper:.3f})"
    
    # Store results in a dictionary
    result = {
        "Feature": df1.columns[col],
        "AUC(CI)": auc_ci,
        "Z_Score": z_score,
        "P_Value": p_value
    }
    results.append(result)
    
    # Print results
    print(f"Feature {col} ({df1.columns[col]}):")
    print(f"  AUC = {auc2:.3f}, 95% CI = [{ci2_lower:.3f}, {ci2_upper:.3f}]")
    print(f"  DeLong Test: Z-score = {z_score:.3f}, P-value = {p_value:.4f}")
    print("-" * 50)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to a CSV file
results_df.to_csv("../table_and_figure/model_comparison_results.csv", index=False)
results_df

Feature 1 (Age):
  AUC = 0.598, 95% CI = [0.480, 0.724]
  DeLong Test: Z-score = -5.043, P-value = 0.0000
--------------------------------------------------
Feature 2 (BMI):
  AUC = 0.670, 95% CI = [0.531, 0.794]
  DeLong Test: Z-score = -3.683, P-value = 0.0002
--------------------------------------------------
Feature 3 (WC):
  AUC = 0.619, 95% CI = [0.474, 0.757]
  DeLong Test: Z-score = -3.739, P-value = 0.0002
--------------------------------------------------
Feature 15 (SBP_bin):
  AUC = 0.531, 95% CI = [0.416, 0.638]
  DeLong Test: Z-score = -5.159, P-value = 0.0000
--------------------------------------------------
Feature 16 (DBP_bin):
  AUC = 0.501, 95% CI = [0.500, 0.503]
  DeLong Test: Z-score = -15.300, P-value = 0.0000
--------------------------------------------------
Feature 17 (TG_bin):
  AUC = 0.527, 95% CI = [0.425, 0.641]
  DeLong Test: Z-score = -6.722, P-value = 0.0000
--------------------------------------------------
Feature 18 (LDL_bin):
  AUC = 0.517, 95% CI 

,Feature,AUC(CI),Z_Score,P_Value
0,Age,"0.60(0.480,0.724)",-5.042618,4.592052e-07
1,BMI,"0.67(0.531,0.794)",-3.683136,2.303821e-04
2,WC,"0.62(0.474,0.757)",-3.739175,1.846248e-04
3,SBP_bin,"0.53(0.416,0.638)",-5.159403,2.477379e-07
4,DBP_bin,"0.50(0.500,0.503)",-15.300448,7.593251e-53
5,TG_bin,"0.53(0.425,0.641)",-6.721876,1.793991e-11
6,LDL_bin,"0.52(0.415,0.624)",-5.577356,2.442018e-08
7,FBG_bin,"0.49(0.477,0.493)",-16.010307,1.082744e-57
8,Renin_bin,"0.83(0.814,0.855)",-2.101466,3.560011e-02
9,PAC_bin_10,"0.59(0.575,0.610)",-10.667287,1.447921e-26


In [4]:
#show the python environment
import session_info

session_info.show()